In [194]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# các thư viện sử dụng
from lxml import html  
import json
import requests
import re
from dateutil import parser as dateparser
from time import sleep
from bs4 import BeautifulSoup

#remove stop word
import nltk
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle') 

# download stopword
#nltk.download('stopwords')
#nltk.download('punkt')

# split() string qith multiple delimiter
import re

In [195]:
def ParseReviews(asin):
    #This script has only been tested with Amazon.com
    amazon_url  = 'http://www.amazon.com/product-reviews/'+asin
    # Add some recent user agent to prevent amazon from blocking the request 
    # Find some chrome user agent strings  here https://udger.com/resources/ua-list/browser-detail?browser=Chrome
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36'}

    for i in range(3):

        response = requests.get(amazon_url,headers = headers,verify=False)
        if response.status_code==404:
            return {"url":amazon_url,"error":"page not found"}
        if response.status_code!=200:
            continue

        '''
        The HTML xpath parser from lxml.etree seems to have max depth limit. 
        It won't traverse further to parse the text if the depth exceeds 254. 
        To avoid this, splitting the html into chunks
        '''
        # chuyển thành cây
        tree = html.fromstring(response.text)
        # gán xpath
        xpath_pageNext = '//li[@data-reftag="cm_cr_arp_d_paging_btm"]'

        # tiến hành lấy thông tin link các trang
        ls_page = tree.xpath(xpath_pageNext)
        links = []
        
        ## lấy hết các trang review của sản phẩm
        for page in ls_page:
            url = 'https://amazon.com/' + page.find('a').get('href')
            links.append(url)
        ## lấy được các trang rồi thì break ngắt vong lập
        break
    
    # lấy reviews ở trang hiện tại
    xpath_reviewsList = '//div[@id="cm_cr-review_list"]'
    xpath_review = '//div[@data-hook="review"]'
    
    review_str = tree.xpath(xpath_reviewsList)
    
    for i in review_str:
        # lấy từng chuỗi review
        reviews = i.xpath(xpath_review)
    
    ## cấu trúc lưu dữ liệu là 1 dic.
    reviews_list = []
    
    ## parse từng chuỗi review:
    for review in reviews:
        
        ls_div = review.find('div').findall('div')
        #### chỉ lấy 4 div đầu tiên theo thứ tự bên dưới
        ### lấy tên
        name = ls_div[0].find('a').findall('div', {'class': 'a-profile-content'})#.findall('div')
        name1 = name[1].find('span', {'class': 'a-profile-avatar-wrapper'}).text
        
        ### đầu tiên là lấy số sao
        star = ls_div[1].find('a').find('i').find('span').text
        ### tiếp theo lấy thời gian
        date = review.find('div').find('span').text
        ### tiếp theo lấy text
        review_content = ls_div[3].find('span').text
        
        ### tiến hành lưu dữ liệu ### tất cả review được chuyển về lower case
        dic_review = {
            'review_author' : name1,
            'review_rating' : star,
            'review_posted_date' : date,
            'review_text' : review_content.lower()
        }
        reviews_list.append(dic_review)
        

    # lấy reviews trong các trang còn lại
    for link in links:
        response = requests.get(link,headers = headers,verify=False)
        
        # tiến hành lấy dữ liệu như trên
        review_str = tree.xpath(xpath_reviewsList)
    
        for i in review_str:
            # lấy từng chuỗi review
            reviews = i.xpath(xpath_review)
        
        ## parse từng chuỗi review:
        for review in reviews:

            ls_div = review.find('div').findall('div')
            #### chỉ lấy 4 div đầu tiên theo thứ tự bên dưới
            ### lấy tên
            name = ls_div[0].find('a').findall('div', {'class': 'a-profile-content'})#.findall('div')
            name1 = name[1].find('span', {'class': 'a-profile-avatar-wrapper'}).text

            ### đầu tiên là lấy số sao
            star = ls_div[1].find('a').find('i').find('span').text
            ### tiếp theo lấy thời gian
            date = review.find('div').find('span').text
            ### tiếp theo lấy text
            review_content = ls_div[3].find('span').text
            
            ### tiến hành lưu dữ liệu ### tất cả review được chuyển về lower case
            dic_review = {
                'review_author' : name1,
                'review_rating' : star,
                'review_posted_date' : date,
                'review_text' : review_content.lower()
            }
            reviews_list.append(dic_review)
            
    return reviews_list

In [196]:
def ReadAsin():
    AsinList = ['B07H5JNWTS', 'B075R9PX3V', 'B075R93J59', 'B073QMF6Z9', 'B075QMZH2L']
    extracted_data = []
    for asin in AsinList:
        print("Downloading and processing page http://www.amazon.com/product-reviews/"+asin)
        reviews = ParseReviews(asin)
        reviews_len = len(reviews)
        dic = {
            'asin' : asin,
            'reviews count' : reviews_len,
            'reviews' : reviews
        }
        extracted_data.append(dic)
        sleep(5)
    f = open('DuLieuTho.json','w')
    json.dump(extracted_data,f,indent=4,ensure_ascii=False)


### đọc dữ liệu lên và tiền xử lí

loại bỏ stopwords

ngắt câu

---

file input: blabla_data.json

file output: preProcData.json

In [197]:
def removeStopWords(review):
    # khởi tạo 1 set stop word
    stop_words = set(stopwords.words('english')) 
    # tách câu thành tokens  
    word_tokens = word_tokenize(review)
    # tra từng tokens trong set stopword và lưu phần còn lại vào lại câu
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    # trả về
    re_review = ' '.join(filtered_sentence)
    return re_review

In [198]:
def preProcReviews( input, output):
    # đọc file và load dữ liệu
    f = open(input,'r')
    data_jsons = json.loads(f.read())
    f.close()
    
    num_pd = len(data_jsons)
    for i_pd in range(num_pd):
        num_reviews = len((data_jsons[i_pd])['reviews'])
        for i_rv in range(num_reviews):
            # tiến hành tách câu sử dụng pickle của nltk
            data_jsons[i_pd]['reviews'][i_rv]['review_text'] = tokenizer.tokenize(data_jsons[i_pd]['reviews'][i_rv]['review_text'], realign_boundaries=True)
            #tiến hành tách từ và loại bỏ stopwords trong mỗi câu
            num_sentences = len(data_jsons[i_pd]['reviews'][i_rv]['review_text'])
            for sentence in range(num_sentences):
                data_jsons[i_pd]['reviews'][i_rv]['review_text'][sentence] = removeStopWords(data_jsons[i_pd]['reviews'][i_rv]['review_text'][sentence])
    # tiến hành lưu lại dữ liệu
    f = open(output,'w')
    json.dump(data_jsons,f,indent=4,ensure_ascii=False)

In [199]:
if __name__ == '__main__':
    ReadAsin()
    preProcReviews('DuLieuTho.json', 'DulieuDaQuaTienXuLi.json')

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTT

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTT

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTT

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTT

C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\ProgramData\Anaconda3\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTT